# Z

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import csv
import datetime as dt
import re
import os
import json
import random
import time,pickle
import scipy.sparse as sp
from operator import itemgetter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
from ast import literal_eval

In [3]:
from scipy import spatial
from math import*
from scipy.spatial import distance

In [41]:
path= "C:/MyPath/Graphs/" #where you saved your data

In [105]:
#hits
G = nx.read_weighted_edgelist(path + "vax_twitter_edgelist.txt" , create_using=nx.Graph(), delimiter=",")
hub_scores, auth_scores= nx.hits(G)

In [106]:
hub_c= pd.DataFrame(hub_scores.items(), columns=['node', 'hscore'])
auth_c= pd.DataFrame(auth_scores.items(), columns=['node', 'ascore'])
perc= round(auth_c.shape[0]*30/100)

In [107]:
hnodes= hub_c.sort_values('hscore', ascending=False)[:perc]['node'].tolist()
anodes= auth_c.sort_values('ascore', ascending=False)[:perc]['node'].tolist()
central= hnodes+anodes

In [108]:
f1 = open("C:/MyPath/vax_twittercommunity0.csv" )
f2 = open("C:/MyPath/vax_twittercommunity1.csv" )

In [110]:
df= pd.read_csv(path+ 'vax_twitter_emb.csv') #data with embeddings

In [160]:
df

,username,text_cleaned,embeddings
0,01_taffy,"by the way, the source of what i said is mitku...","[-0.20642881095409393, -0.029854411259293556, ..."
1,0_politics2,anti vaccination? nope not at all,"[-0.2774412930011749, 0.033726152032613754, 0...."
2,102_virus,"what is a vaccination competition? anyway, you...","[0.072479248046875, 0.0009308835724368691, 0.0..."
3,10queues,anti-vaccine activist livestreams himself shov...,"[-0.20346961915493011, 0.3762381076812744, 0.0..."
4,10tv,kentucky students lose appeal in chickenpox va...,"[-0.1136634424328804, 0.10152998566627502, 0.1..."
...,...,...,...
12451,zumiprep,anti-vaccination group calls for doj investiga...,"[-0.15897351503372192, 0.17765064537525177, 0...."
12452,zuzyque123,"i’m sorry mad, i scrolled forever trying to fi...","[-0.09513148665428162, 0.2753380537033081, 0.0..."
12453,zylon9,you seem to be saying that vaccination can cau...,"[-0.1944509595632553, -0.07257141172885895, 0...."
12454,zynks,"have you started the ""church of vaccination pu...","[-0.15605027973651886, -0.02449946105480194, -..."


In [111]:
lines1 = f1.readlines()
lines1= list(filter(('\n').__ne__, lines1 ))
dict_left = {}

for line in lines1:
    line = line.strip();
    dict_left[line] = 1;
    
lines2 = f2.readlines()
lines2= list(filter(('\n').__ne__, lines2 ))
dict_right = {}

for line in lines2:
    line = line.strip();
    dict_right[line] = 1;

In [112]:
com1= set(dict_left.keys())
com2= set(dict_right.keys())

In [114]:
emb1= df[df['username'].isin(com1) & df['username'].isin(central) ]['embeddings']
emb2= df[df['username'].isin(com2) & df['username'].isin(central)]['embeddings']
embeddings= df['embeddings']

In [115]:
c1=[]
for x in emb1:
    c1.append(literal_eval(x))
c1= np.array(c1)
    
c2=[]
for x in emb2:
    c2.append(literal_eval(x))
c2= np.array(c2)

X=[]
for x in embeddings:
    X.append(literal_eval(x))
X= np.array(X)
    

In [116]:
# calculate centroids
cent1 = c1.mean(axis=0)
cent2 = c2.mean(axis=0)
cent0 = X.mean(axis=0)


In [117]:
v = np.cov(X.T)
SS0 = 0
for row in X:
    SS0 = SS0 + distance.cosine(row,cent0)
    
v = np.cov(c1.T)
SS1 = 0
for row in c1:
    SS1 = SS1 + distance.cosine(row,cent1)
    
v = np.cov(c2.T)
SS2 = 0
for row in c2:
    SS2 = SS2 + distance.cosine(row,cent2)



In [ ]:
# Controversy index
idx = (SS1+SS2)/SS0
print(round(idx,4))
print(round(1- round(idx,4),4)) # 1- z